In [ ]:
!pip -q install unsloth
!pip install -U unsloth_zoo
!pip install -U --no-deps git+https://github.com/unslothai/unsloth.git

!pip -q install "trl>=0.9.6" "datasets>=2.20.0" "accelerate>=0.33.0" "bitsandbytes>=0.43.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive


import torch
print("="*60)
print("🔧 SISTEM BİLGİSİ")
print("="*60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram:.1f} GB")


🔧 SISTEM BİLGİSİ
PyTorch: 2.9.1+cu128
CUDA: True
GPU: NVIDIA A100-SXM4-80GB
VRAM: 85.2 GB


In [ ]:
import os

# ✅ Veri yolu (Drive)
DATA_DIR = "/content/drive/MyDrive/verimodel2"
TRAIN_FILE = os.path.join(DATA_DIR, "train.jsonl")
VALID_FILE = os.path.join(DATA_DIR, "valid.jsonl")

assert os.path.exists(TRAIN_FILE), f"Train yok: {TRAIN_FILE}"
assert os.path.exists(VALID_FILE), f"Valid yok: {VALID_FILE}"

# ✅ Model
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct"

# Küçük veri için 1024 genelde yeterli (hem hızlı, hem overfit riski daha düşük)
MAX_SEQ_LENGTH = 1536

# ✅ Overfit’i azaltan ana kararlar:
# - epoch düşük
# - LR makul ama aşırı değil
# - LoRA r küçük/orta
# - dropout biraz daha yüksek
NUM_EPOCHS = 6
LEARNING_RATE = 2e-5

# A100’de rahat: 4bit + grad_accum ile efektif batch büyüt
BATCH_SIZE = 4
GRAD_ACCUM = 8  # efektif batch = 64

OUTPUT_DIR = "./otel-ozet-llama31-8b-lora6epoch"


In [ ]:
import torch
from unsloth import FastLanguageModel

print("="*60)
print("📦 MODEL YÜKLENİYOR")
print("="*60)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=torch.bfloat16,
    load_in_4bit=True,          # ✅ VRAM düşürür, küçük veri için ideal
)

print("✅ Model yüklendi:", MODEL_NAME)

print("="*60)
print("🔧 LoRA AYARLANIYOR")
print("="*60)

# Küçük veri → r çok büyütme (64 gibi) daha kolay overfit yapar.
model = FastLanguageModel.get_peft_model(
    model,
    r=16,                       # ✅ küçük veri için güvenli
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
    ],
    lora_alpha=32,              # genelde ~2*r iyi gider
    lora_dropout=0.10,          # ✅ overfit’e karşı arttırdık
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

model.print_trainable_parameters()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
📦 MODEL YÜKLENİYOR
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


✅ Model yüklendi: unsloth/Meta-Llama-3.1-8B-Instruct
🔧 LoRA AYARLANIYOR


Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
print("✅ LLaMA 3.1 chat template ayarlandı!")


✅ LLaMA 3.1 chat template ayarlandı!


In [ ]:
from datasets import load_dataset

print("="*60)
print("📂 VERİ YÜKLENİYOR")
print("="*60)

data_files = {"train": TRAIN_FILE, "validation": VALID_FILE}
ds = load_dataset("json", data_files=data_files)

train_dataset = ds["train"]
val_dataset   = ds["validation"]

print("✅ Yüklendi")
print("Train:", len(train_dataset))
print("Valid:", len(val_dataset))

print("\n📝 Örnek (messages ilk 2):")
print(train_dataset[0]["messages"][:2])


📂 VERİ YÜKLENİYOR


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

✅ Yüklendi
Train: 504
Valid: 56

📝 Örnek (messages ilk 2):
[{'role': 'system', 'content': 'Sen otel yorumlarından çıkarılan aspect değerlendirmelerine göre, otel için Türkçe genel bir özet yazan bir asistansın.\nKurallar:\n- Sayısal değerleri (pozitif/negatif sayıları vb.) aynen yazma, sayısız/niteliksel anlat.\n- Güçlü yönleri ve geliştirilmesi gereken alanları dengeli şekilde belirt.\n- 3-4 paragraf, akıcı ve doğal bir dil kullan.\n- Gereksiz tekrar yapma.\n'}, {'role': 'user', 'content': 'Otel: GDZ Gediz Business Gediz\n\nAspect değerlendirmeleri:\n- Aktivite olanakları: Karışık (övülen: olumlu_kalite)\n- Balkon / teras: Karışık (övülen: kalite)\n- Banyo ve tuvalet: Ağırlıklı olumsuz (övülen: kalite | şikayet: kalite, yokluk)\n- Fitness / spor: Karışık (övülen: kalite)\n- Fiyat-performans: Karışık (övülen: fiyat | şikayet: fiyat)\n- Sessizlik / gürültü durumu: Çok olumlu (övülen: kalite, erişim)\n- Güvenlik: Karışık (övülen: kalite)\n- Klima / ısıtma: Çok olumsuz (övülen: erişim | ş

In [ ]:
print("="*60)
print("🔄 VERİ FORMATLANIYOR (ASSISTANT-ONLY MASKING / PREFIX LENGTH)")
print("="*60)

def formatting_func(examples):
    conversations = examples["messages"]

    input_ids_list = []
    labels_list = []

    for convo in conversations:
        # 1) full konuşma tokenları
        full_ids = tokenizer.apply_chat_template(
            convo,
            tokenize=True,
            add_generation_prompt=False,
        )

        # 2) prefix = system + user (assistant hariç)
        prefix_msgs = [m for m in convo if m["role"] != "assistant"]
        prefix_ids = tokenizer.apply_chat_template(
            prefix_msgs,
            tokenize=True,
            add_generation_prompt=False,
        )

        labels = full_ids.copy()

        # 3) prefix uzunluğu kadar mask
        cut = min(len(prefix_ids), len(labels))
        for i in range(cut):
            labels[i] = -100

        # (Güvenlik) Eğer hâlâ tümü -100 ise, debug için küçük ipucu bırak
        if all(x == -100 for x in labels):
            # Çok nadir: şablon uyumsuzluğu / veri bozukluğu
            # Bu örneği tamamen maskli bırakmamak için en sona küçük bir kısmı açalım
            # (Ama istersen burada raise da edebilirsin)
            if len(labels) > 8:
                for i in range(len(labels)-8, len(labels)):
                    labels[i] = full_ids[i]

        input_ids_list.append(full_ids)
        labels_list.append(labels)

    return {"input_ids": input_ids_list, "labels": labels_list}

train_dataset = train_dataset.map(
    formatting_func,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Train masking"
)

val_dataset = val_dataset.map(
    formatting_func,
    batched=True,
    remove_columns=val_dataset.column_names,
    desc="Valid masking"
)

print("✅ Masking tamamlandı")

# 🧪 Kontrol: örnekte kaç token label aktif?
def active_label_count(ex):
    return sum(1 for x in ex["labels"] if x != -100)

print("\n🧪 Kontrol")
print("Train[0] aktif label token:", active_label_count(train_dataset[0]))
print("Valid[0] aktif label token:", active_label_count(val_dataset[0]))
print("Train[0] ilk 40 label:", train_dataset[0]["labels"][:40])


🔄 VERİ FORMATLANIYOR (ASSISTANT-ONLY MASKING / PREFIX LENGTH)


Train masking:   0%|          | 0/504 [00:00<?, ? examples/s]

Valid masking:   0%|          | 0/56 [00:00<?, ? examples/s]

✅ Masking tamamlandı

🧪 Kontrol
Train[0] aktif label token: 378
Valid[0] aktif label token: 381
Train[0] ilk 40 label: [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

print("="*60)
print("⚙️ TRAINER AYARLANIYOR")
print("="*60)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # ✅ Küçük veri → az epoch
    num_train_epochs=NUM_EPOCHS,

    # Batch
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,

    # LR
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.10,                 # küçük veri → biraz daha yüksek warmup iyi gelir

    # Precision
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),

    # Eval + save (sık kontrol → overfit yakala)
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=20,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Regularization
    weight_decay=0.05,                 # küçük veri → biraz arttırdık
    max_grad_norm=0.3,

    # Optimizasyon
    optim="adamw_8bit",

    # Logging
    logging_steps=10,
    logging_first_step=True,
    report_to="none",

    # Stabil
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field=None,
    packing=False,
    max_seq_length=MAX_SEQ_LENGTH,



    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # ✅ eval_loss düzelmezse dur
)

print("✅ Trainer hazır")


⚙️ TRAINER AYARLANIYOR
✅ Trainer hazır


In [ ]:
import torch
print("="*60)
print("🚀 EĞİTİM BAŞLIYOR")
print("="*60)

gpu_stats = torch.cuda.get_device_properties(0)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU: {gpu_stats.name} | VRAM: {max_memory} GB")
print(f"Epochs: {NUM_EPOCHS} | LR: {LEARNING_RATE}")
print(f"Batch: {BATCH_SIZE} x {GRAD_ACCUM} = {BATCH_SIZE*GRAD_ACCUM}")
print("-"*60)

trainer_stats = trainer.train()

print("✅ Eğitim bitti")
print(trainer_stats.metrics)


The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 EĞİTİM BAŞLIYOR
GPU: NVIDIA A100-SXM4-80GB | VRAM: 79.318 GB
Epochs: 6 | LR: 2e-05
Batch: 4 x 8 = 32
------------------------------------------------------------


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 504 | Num Epochs = 6 | Total steps = 96
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,1.514300,1.406771
40,1.148800,1.111071
60,0.986900,0.989128
80,0.929600,0.950914


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


✅ Eğitim bitti
{'train_runtime': 1022.3461, 'train_samples_per_second': 2.958, 'train_steps_per_second': 0.094, 'total_flos': 1.4331755359843123e+17, 'train_loss': 1.1480924375355244, 'epoch': 6.0}


In [ ]:
import shutil, os

MERGED_DIR = "/content/otel_ozet_llama31_8b_merged"
if os.path.exists(MERGED_DIR):
    shutil.rmtree(MERGED_DIR)
os.makedirs(MERGED_DIR, exist_ok=True)

print("✅ Temizlendi:", MERGED_DIR)


✅ Temizlendi: /content/otel_ozet_llama31_8b_merged


In [ ]:
import torch, os
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "unsloth/Meta-Llama-3.1-8B-Instruct"
LORA_DIR   = "/content/otel-ozet-llama31-8b-lora6epoch/checkpoint-80"
MERGED_DIR = "/content/otel_ozet_llama31_8b_merged"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

# Merge için 4bit KULLANMA: fp16/bf16 yükle
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# LoRA adapter'ı bağla
model = PeftModel.from_pretrained(model, LORA_DIR)
print("✅ Base + LoRA yüklendi")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Base + LoRA yüklendi


In [ ]:
import os

print("🔨 merge_and_unload başlıyor...")
merged_model = model.merge_and_unload()
print("✅ Merge tamam")

print("💾 Kaydediliyor...")
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("✅ Kaydedildi:", MERGED_DIR)
print("📁 İçerik:", os.listdir(MERGED_DIR)[:30])


🔨 merge_and_unload başlıyor...
✅ Merge tamam
💾 Kaydediliyor...
✅ Kaydedildi: /content/otel_ozet_llama31_8b_merged
📁 İçerik: ['chat_template.jinja', 'tokenizer_config.json', 'special_tokens_map.json', 'model.safetensors.index.json', 'model-00002-of-00004.safetensors', 'config.json', 'generation_config.json', 'model-00001-of-00004.safetensors', 'tokenizer.json', 'model-00003-of-00004.safetensors', 'model-00004-of-00004.safetensors']


In [ ]:
import torch
from unsloth import FastLanguageModel

MODEL_DIR = "/content/otel_ozet_llama31_8b_merged"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=1536,
    dtype=torch.bfloat16,
    load_in_4bit=True,   # inference için çok iyi
)
FastLanguageModel.for_inference(model)

test_messages = [
    {
        "role": "system",
        "content": "Sen otel yorumlarından çıkarılan aspect değerlendirmelerine göre, otel için Türkçe genel bir özet yazan bir asistansın.\nKurallar:\n- Sayısal değerleri (pozitif/negatif sayıları vb.) aynen yazma, sayısız/niteliksel anlat.\n- Güçlü yönleri ve geliştirilmesi gereken alanları dengeli şekilde belirt.\n- 3-4 paragraf, akıcı ve doğal bir dil kullan.\n- Gereksiz tekrar yapma.\n"
    },
    {
        "role": "user",
        "content": "Otel: Ramada by Wyndham Ankara Kavaklıdere\n\nAspect değerlendirmeleri:\n- Banyo ve tuvalet: Çok olumsuz (şikayet: kalite, yokluk)\n- Fiyat-performans: Karışık (övülen: fiyat | şikayet: fiyat, kalite)\n- Sessizlik / gürültü durumu: Karışık (övülen: erişim, kalite | şikayet: erişim)\n- Güvenlik: Çok olumlu (övülen: kalite)\n- Havuz: Karışık (şikayet: kalite)\n- Klima / ısıtma: Çok olumsuz (şikayet: kalite)\n- Konum: Çok olumlu (övülen: erişim, kalite)\n- Oda kalitesi: Ağırlıklı olumlu (övülen: kalite | şikayet: kalite)\n- Otopark: Çok olumsuz (övülen: erişim | şikayet: yokluk, erişim)\n- Personel: Çok olumlu (övülen: servis, olumlu_kalite | şikayet: servis)\n- Resepsiyon: Karışık (övülen: servis, kalite | şikayet: servis)\n- Spa / hamam: Karışık (şikayet: yokluk)\n- Temizlik: Ağırlıklı olumlu (övülen: kalite | şikayet: kalite, servis)\n- Uyku ve yatak konforu: Ağırlıklı olumsuz (övülen: kalite | şikayet: kalite)\n- Yemek kalitesi: Çok olumlu (övülen: kalite, servis | şikayet: kalite)\n- Yemek çeşitliliği: Çok olumlu (övülen: kalite)\n\nAşağıdaki aspect değerlendirmelerine göre otel için genel bir özet yaz.\nNot: Sayısal değerleri metne taşımadan, niteliksel ifadelerle anlat.\n"
    }
]

input_ids = tokenizer.apply_chat_template(
    test_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

gen_kwargs = dict(
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.08,
)

with torch.no_grad():
    out = model.generate(input_ids, **gen_kwargs)

prompt_len = input_ids.shape[-1]
gen_text = tokenizer.decode(out[0][prompt_len:], skip_special_tokens=True)

print("======== MODEL ÇIKTISI ========")
print(gen_text.strip())


==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# -*- coding: utf-8 -*-
"""GGUF Dönüşüm - otel_ozet_llama31_8b_merged (Düzeltilmiş)"""

# 1. Transformers versiyonunu düzelt
!pip install -q transformers==4.57.3 accelerate safetensors sentencepiece

# 2. llama.cpp'yi klonla ve CMAKE ile derle
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && mkdir -p build && cd build && cmake .. && cmake --build . --config Release -j

# 3. Model yolunu kontrol et
import os
MODEL_DIR = "/content/otel_ozet_llama31_8b_merged"

if not os.path.exists(MODEL_DIR):
    print(f"❌ HATA: {MODEL_DIR} bulunamadı!")
    print("📁 Drive içeriğini kontrol ediyorum...")
    !ls -la /content/drive/MyDrive/ | grep otel
else:
    print(f"✅ Model klasörü bulundu: {MODEL_DIR}")
    print("\n📂 İçindeki dosyalar:")
    !ls -lh {MODEL_DIR}

# 4. HuggingFace modelini F16 GGUF formatına dönüştür
!python llama.cpp/convert_hf_to_gguf.py {MODEL_DIR} \
    --outfile /content/otel_ozet_model.f16.gguf \
    --outtype f16

# 5. Dönüşümü kontrol et
if os.path.exists("/content/otel_ozet_model.f16.gguf"):
    print("✅ F16 GGUF oluşturuldu")
    !ls -lh /content/otel_ozet_model.f16.gguf

    # 6. F16'dan Q4_K_M'e quantize et
    !./llama.cpp/build/bin/llama-quantize \
        /content/otel_ozet_model.f16.gguf \
        /content/otel_ozet_model.Q4_K_M.gguf \
        Q4_K_M

    # 7. Q4 modelini kontrol et
    if os.path.exists("/content/otel_ozet_model.Q4_K_M.gguf"):
        print("\n" + "="*60)
        print("✅ DÖNÜŞÜM TAMAMLANDI!")
        print("="*60)
        !ls -lh /content/otel_ozet_model.*.gguf

        # 8. Google Drive'a kopyala
        import shutil
        shutil.copy2("/content/otel_ozet_model.Q4_K_M.gguf",
                     "/content/drive/MyDrive/")
        print("\n✅ Q4 model Google Drive'a kopyalandı")
        !ls -lh /content/drive/MyDrive/otel_ozet_model.Q4_K_M.gguf
    else:
        print("❌ Q4 quantization başarısız!")
else:
    print("❌ F16 GGUF oluşturulamadı!")

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
CMAKE_BUILD_TYPE=Release
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- x86 detected
-- Adding CPU backend variant ggml-cpu: -march=native 
-- ggml version: 0.9.5
-- ggml commit:  3e4bb2966
-- Generating embedded license file for target: common
-- Configuring done (0.6s)
-- Generating done (0.3s)
-- Build files have been written to: /content/llama.cpp/build
[  0%] Built target build_info
[  0%] Built target sha256
[  1%] Built target xxhash
[  1%] Built target llama-llava-cli
[  2%] Built target llama-minicpmv-cli
[  3%] Built target sha1
[  4%] Built target llama-gemma3-cli
[  4%] Built target llama-qwen2vl-cli
[  4%] Built target cpp-httplib
[  6%] Built target ggml-base
[ 10%] Built target ggml-cpu
[ 11%] Built target ggml
[ 11%] Built target 

In [ ]:
# -*- coding: utf-8 -*-
"""GGUF Model Test - Olumlu Otel Örneği"""

from llama_cpp import Llama

# Model yükle
MODEL_PATH = "/content/drive/MyDrive/otel_ozet_model.Q4_K_M.gguf"

print("🔄 Model yükleniyor...")
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=0,
)
print("✅ Model yüklendi!\n")

# System prompt
system_prompt = """Sen otel yorumlarından çıkarılan aspect değerlendirmelerine göre, otel için Türkçe genel bir özet yazan bir asistansın.
Kurallar:
- Sayısal değerleri (pozitif/negatif sayıları vb.) aynen yazma, sayısız/niteliksel anlat.
- Güçlü yönleri ve geliştirilmesi gereken alanları dengeli şekilde belirt.
- 3-4 paragraf, akıcı ve doğal bir dil kullan.
- Gereksiz tekrar yapma."""

# Test 1: Lüks Resort - Çok Olumlu
test1 = {
    "otel_adi": "Maxx Royal Belek Golf Resort",
    "aspects": """- Aktivite olanakları: Çok olumlu (övülen: kalite, çeşitlilik)
- Banyo ve tuvalet: Çok olumlu (övülen: kalite, lüks)
- Fiyat-performans: Ağırlıklı olumlu (övülen: kalite | şikayet: fiyat)
- Sessizlik / gürültü durumu: Çok olumlu (övülen: kalite, huzur)
- Güvenlik: Çok olumlu (övülen: kalite, profesyonellik)
- Havuz: Çok olumlu (övülen: kalite, temizlik, çeşitlilik)
- İnternet / Wi-Fi: Çok olumlu (övülen: hız, erişim)
- Klima / ısıtma: Çok olumlu (övülen: kalite, ayarlanabilirlik)
- Konum: Çok olumlu (övülen: deniz, erişim, manzara)
- Manzara: Çok olumlu (övülen: kalite, muhteşem)
- Oda kalitesi: Çok olumlu (övülen: genişlik, lüks, tasarım)
- Oda servisi: Çok olumlu (övülen: hız, kalite, servis)
- Otopark: Çok olumlu (övülen: vale, erişim)
- Personel: Çok olumlu (övülen: servis, ilgi, güleryüz, profesyonellik)
- Resepsiyon: Çok olumlu (övülen: servis, hız, çözüm odaklı)
- Spa / hamam: Çok olumlu (övülen: kalite, çeşitlilik, lüks)
- Temizlik: Çok olumlu (övülen: kalite, titizlik, günlük)
- Uyku ve yatak konforu: Çok olumlu (övülen: kalite, konfor, yorgan)
- Yemek kalitesi: Çok olumlu (övülen: kalite, lezzet, taze)
- Yemek çeşitliliği: Çok olumlu (övülen: çeşitlilik, a la carte, büfe)"""
}

# Test 2: Butik Otel - Dengeli Olumlu
test2 = {
    "otel_adi": "Hera Hotel Istanbul",
    "aspects": """- Banyo ve tuvalet: Çok olumlu (övülen: temizlik, modern)
- Fiyat-performans: Çok olumlu (övülen: fiyat, kalite)
- Sessizlik / gürültü durumu: Ağırlıklı olumlu (övülen: huzur | şikayet: cadde_gürültüsü)
- Güvenlik: Çok olumlu (övülen: güvenli, 7/24)
- İnternet / Wi-Fi: Çok olumlu (övülen: hız, ücretsiz)
- Klima / ısıtma: Ağırlıklı olumlu (övülen: kalite)
- Konum: Çok olumlu (övülen: merkezi, Sultanahmet, ulaşım)
- Manzara: Ağırlıklı olumlu (övülen: tarihi, Ayasofya)
- Oda kalitesi: Çok olumlu (övülen: temiz, şık, rahat)
- Personel: Çok olumlu (övülen: servis, yardımsever, samimi, Türkçe/İngilizce)
- Resepsiyon: Çok olumlu (övülen: hız, bilgilendirme, tur_önerileri)
- Temizlik: Çok olumlu (övülen: titiz, günlük)
- Uyku ve yatak konforu: Çok olumlu (övülen: rahat, kaliteli)
- Yemek kalitesi: Çok olumlu (övülen: kahvaltı, ev_yapımı)
- Yemek çeşitliliği: Ağırlıklı olumlu (övülen: kahvaltı | şikayet: akşam_yemeği_yok)"""
}

# Test 3: Termal Otel - Dengeli
test3 = {
    "otel_adi": "Richmond Pamukkale Thermal Hotel",
    "aspects": """- Aktivite olanakları: Ağırlıklı olumlu (övülen: termal, animasyon)
- Banyo ve tuvalet: Çok olumlu (övülen: temiz, geniş)
- Fiyat-performans: Çok olumlu (övülen: uygun, termal_dahil)
- Sessizlik / gürültü durumu: Ağırlıklı olumlu (övülen: sakin)
- Havuz: Çok olumlu (övülen: termal, sıcak, temiz)
- Klima / ısıtma: Ağırlıklı olumlu (övülen: ısıtma | şikayet: klima_eski)
- Konum: Çok olumlu (övülen: Pamukkale_yakın, doğa)
- Manzara: Çok olumlu (övülen: dağ, yeşillik)
- Oda kalitesi: Ağırlıklı olumlu (övülen: geniş, temiz | şikayet: mobilya_eski)
- Personel: Çok olumlu (övülen: güler_yüzlü, ilgili)
- Resepsiyon: Ağırlıklı olumlu (övülen: servis)
- Spa / hamam: Çok olumlu (övülen: termal, masaj, bakım)
- Temizlik: Çok olumlu (övülen: temiz)
- Uyku ve yatak konforu: Ağırlıklı olumlu (övülen: rahat)
- Yemek kalitesi: Ağırlıklı olumlu (övülen: lezzet, sıcak | şikayet: tuzsuz)
- Yemek çeşitliliği: Ağırlıklı olumlu (övülen: çeşitli)"""
}

# Test fonksiyonu
def test_hotel(test_data, test_no):
    user_prompt = f"""Otel: {test_data['otel_adi']}

Aspect değerlendirmeleri:
{test_data['aspects']}

Aşağıdaki aspect değerlendirmelerine göre otel için genel bir özet yaz.
Not: Sayısal değerleri metne taşımadan, niteliksel ifadelerle anlat."""

    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    print("="*70)
    print(f"🏨 TEST {test_no}: {test_data['otel_adi']}")
    print("="*70)
    print("\n🔄 Özet üretiliyor...\n")

    output = llm(
        prompt,
        max_tokens=1024,
        temperature=0.7,
        top_p=0.9,
        repeat_penalty=1.1,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        echo=False
    )

    generated = output['choices'][0]['text'].strip()

    print("📝 MODEL ÇIKTISI:")
    print("-"*70)
    print(generated)
    print("-"*70)
    print(f"📊 Token: {output['usage']['completion_tokens']} | Süre: ~{output['usage']['completion_tokens']/20:.1f}s")
    print("="*70)
    print("\n")

# Testleri çalıştır
test_hotel(test1, 1)
test_hotel(test2, 2)
test_hotel(test3, 3)

print("\n✅ TÜM TESTLER TAMAMLANDI!")

🔄 Model yükleniyor...


llama_model_loader: loaded meta data with 31 key-value pairs and 292 tensors from /content/drive/MyDrive/otel_ozet_model.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_p f32              = 0.900000
llama_model_loader: - kv   3:                      general.sampling.temp f32              = 0.600000
llama_model_loader: - kv   4:                               general.name str              = Otel_Ozet_Llama31_8B_Merged
llama_model_loader: - kv   5:                         general.size_label str              = 8.0B
llama_model_loader: - kv   6:                          llama.block_count u32              = 32
llama_model_loader: - kv   7:   

✅ Model yüklendi!

🏨 TEST 1: Maxx Royal Belek Golf Resort

🔄 Özet üretiliyor...



llama_perf_context_print:        load time =   23022.08 ms
llama_perf_context_print: prompt eval time =   23021.30 ms /   616 tokens (   37.37 ms per token,    26.76 tokens per second)
llama_perf_context_print:        eval time =   52658.07 ms /   247 runs   (  213.19 ms per token,     4.69 tokens per second)
llama_perf_context_print:       total time =   76487.75 ms /   863 tokens
llama_perf_context_print:    graphs reused =        239
Llama.generate: 125 prefix-match hit, remaining 392 prompt tokens to eval


📝 MODEL ÇIKTISI:
----------------------------------------------------------------------
Maxx Royal Belek Golf Resort, konuklarından genel olarak yüksek memnuniyet alan ve birçok alanda mükemmel bir hizmet sunan bir otel. Konuklar otelin sunduğu lüks ve konforlu ortamı takdir ediyor. Otelin fiziksel özelliklerinin yanı sıra personeli de misafirlerin en çok övülen tarafı. 

Otel, güzergahının avantajı ile deniz manzarası sunuyor ve bu durum konuklar için büyük bir artı oluşturuyor. Ayrıca, sundugu aktivite çeşitliliği, otopark imkanları ve güvenlik anlayışı da olumlu bir izlenim bırakıyor. Yemeklerin lezzeti ve çeşitliliği de benzer şekilde sıklıkla övülüyor. Temizlik anlayışının titizliği, otelin genel kalitesi konusunda önemli bir referans noktası oluyor.

Fiyat-performans dengesinde ise bazı konuklar beklentilerini karşılamadığını belirtiyor; bu durum, genel memnuniyeti gölgelemekle birlikte, otelin sunduğu hizmet kalitesine rağmen bir şikayet noktası olarak öne çıkıyor.
-------------

llama_perf_context_print:        load time =   23022.08 ms
llama_perf_context_print: prompt eval time =   14018.86 ms /   392 tokens (   35.76 ms per token,    27.96 tokens per second)
llama_perf_context_print:        eval time =   73944.64 ms /   352 runs   (  210.07 ms per token,     4.76 tokens per second)
llama_perf_context_print:       total time =   89144.48 ms /   744 tokens
llama_perf_context_print:    graphs reused =        340
Llama.generate: 125 prefix-match hit, remaining 407 prompt tokens to eval


📝 MODEL ÇIKTISI:
----------------------------------------------------------------------
Hera Hotel Istanbul, misafirlerine sunduğu üstün hizmet kalitesi ve konuklarını memnun eden birçok özellikleriyle öne çıkıyor. Otelin genel atmosferi huzurlu ve rahat bir ortam sunuyor. Personel çalışanlarının yardımsever tutumu, misafirlerin en çok takdir ettiği noktalardan biri olarak öne çıkıyor.

Oda ve banyo tesislerinin temizliği ve modern donanımlı olması büyük beğeni topluyor. Fiyat-performans dengesi de oldukça başarılı bulunmuş ve bu da konuklar için bir artı oluşturuyor. Otelin konumu, Sultanahmet'in merkezinde olmasıyla tarihsel güzelliklere ve kolay ulaşım imkanı sunuyor.

Yemek kalitesi, özellikle kahvaltıda mükemmel bulunuyor; ev yapımı lezzetli options ile misafirler beğeni kazanıyor. İnternet hizmeti ve klima/ısınma sistemleri de beklentileri karşılamakta oldukça başarılı. Güvenlik hizmetleri ve genel temizlik standartları da üst seviyede tutuluyor.

Bir iki küçük nokta ise geliştir

llama_perf_context_print:        load time =   23022.08 ms
llama_perf_context_print: prompt eval time =   14421.22 ms /   407 tokens (   35.43 ms per token,    28.22 tokens per second)
llama_perf_context_print:        eval time =   89833.60 ms /   427 runs   (  210.38 ms per token,     4.75 tokens per second)
llama_perf_context_print:       total time =  105709.55 ms /   834 tokens
llama_perf_context_print:    graphs reused =        413


📝 MODEL ÇIKTISI:
----------------------------------------------------------------------
Richmond Pamukkale Thermal Hotel, konuklarından genel anlamda çok yüksek memnuniyet alan ve birçok alanda övgü toplayan bir tesis. Otelin sunduğu hizmetler ve olanaklar misafirlerin büyük çoğunluğunu tatmin ediyor. Personel hizmetleri, termal oda ve spa gibi imkanlar, temizlik standartları ve konumu konukların en çok takdir ettiği yönler arasında yer alıyor.

Otel personeli güler yüzlü, ilgili ve yardımsever bir tutumla misafirlerini iyi karşılamakta başarılı. Termal oda deneyimi de büyük beğeni topluyor; sıcaklık, temizlik ve genel konforu misafirlere çok memnun bırakıyor. Otelin konumu da oldukça avantajlı, doğa ile iç içe ve Pamukkale'ye yakınlığı övülüyor.

Yemekler konusunda lezzetli ve sıcak sunulması dikkat çekiyor, çeşitli seçeneklerin bulunması ise misafirlerin tatminini artırıyor. Yatakların rahatlığı da konfor açısından önemli bir güçlü yön. Havuz, spa ve hamam hizmetleri de misafirlerin 